In [7]:
# Setup: Add project root to path and import utilities
import sys
sys.path.insert(0, '..')

from utils import get_input
import networkx as nx
import numpy as np
from math import prod

# Load input data for day 8
data = get_input(8)

print(f"Loaded {len(data)} lines of input")
print("Sample data:")
for line in data[:5]:
    print(f"  {line}")

Loaded 1000 lines of input
Sample data:
  27558,61383,12726
  15513,81970,25554
  24379,89821,82524
  42987,15460,38773
  10680,2978,15903


In [8]:
"""Parse junction boxes as numpy array for easy computation."""
boxes = np.array([list(map(int, line.split(','))) for line in data])

print(f"Created {len(boxes)} junction boxes")
print(f"Shape: {boxes.shape}")
print(f"Sample boxes:\n{boxes[:3]}")

Created 1000 junction boxes
Shape: (1000, 3)
Sample boxes:
[[27558 61383 12726]
 [15513 81970 25554]
 [24379 89821 82524]]


In [9]:
"""Create a complete graph with all boxes as nodes and distances as edge weights."""
G = nx.Graph()

# Add nodes with their coordinates
for i, (x, y, z) in enumerate(boxes):
    G.add_node(i, x=x, y=y, z=z)

# Add edges with Euclidean distances as weights
for i in range(len(boxes)):
    for j in range(i + 1, len(boxes)):
        distance = np.linalg.norm(boxes[i] - boxes[j])
        G.add_edge(i, j, weight=distance)

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Get all edges sorted by weight
edges_sorted = sorted(G.edges(data=True), key=lambda x: x[2]['weight'])
print(f"\nShortest edge: {edges_sorted[0][0]} <-> {edges_sorted[0][1]}, distance={edges_sorted[0][2]['weight']:.2f}")
print(f"Longest edge: {edges_sorted[-1][0]} <-> {edges_sorted[-1][1]}, distance={edges_sorted[-1][2]['weight']:.2f}")

Graph: 1000 nodes, 499500 edges

Shortest edge: 42 <-> 801, distance=439.44
Longest edge: 103 <-> 740, distance=161528.59

Shortest edge: 42 <-> 801, distance=439.44
Longest edge: 103 <-> 740, distance=161528.59


## Part 1: Activate shortest 1000 connections

Create a subgraph with only the 1000 shortest edges, then find connected components.

In [10]:
"""Build subgraph with 1000 shortest edges."""
G1 = nx.Graph()
G1.add_nodes_from(G.nodes(data=True))
G1.add_edges_from(edges_sorted[:1000])

print(f"Subgraph: {G1.number_of_nodes()} nodes, {G1.number_of_edges()} edges")

# Find connected components (these are the circuits)
components = list(nx.connected_components(G1))
component_sizes = sorted([len(c) for c in components], reverse=True)

print(f"\nNumber of circuits: {len(components)}")
print(f"Three largest circuit sizes: {component_sizes[:3]}")

Subgraph: 1000 nodes, 1000 edges

Number of circuits: 276
Three largest circuit sizes: [42, 38, 33]


In [11]:
"""Calculate product of three largest circuit sizes."""
result = prod(component_sizes[:3])
print(f"Product: {result}")
result

Product: 52668


52668

## Part 2: Find minimum spanning tree completion

Add edges one by one until the graph is fully connected (one component).

In [12]:
"""Add shortest edges until graph is fully connected."""
G2 = nx.Graph()
G2.add_nodes_from(G.nodes(data=True))

last_edge = None
for edge in edges_sorted:
    G2.add_edge(edge[0], edge[1], **edge[2])
    
    # Check if graph is fully connected (only 1 component)
    if nx.number_connected_components(G2) == 1:
        last_edge = edge
        break

if last_edge:
    node1, node2 = last_edge[0], last_edge[1]
    x1 = G2.nodes[node1]['x']
    x2 = G2.nodes[node2]['x']
    result = x1 * x2
    
    print(f"Last edge added: {node1} <-> {node2}")
    print(f"Distance: {last_edge[2]['weight']:.2f}")
    print(f"Coordinates: ({x1}, ...) and ({x2}, ...)")
    print(f"Product of X coordinates: {result}")
else:
    print("Could not connect graph")
    result = None

result

Last edge added: 410 <-> 942
Distance: 15613.94
Coordinates: (37565, ...) and (39240, ...)
Product of X coordinates: 1474050600


np.int64(1474050600)